In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from Unet_block import Unet
from torch_utils import get_param_num

In [2]:
def conv1x1_bn_relu(inp_activation, output_activation, BN=True, activation = True):
    "con 1x1 + Batchnormalization + relu"
    layer = [nn.Conv2d(inp_activation, output_activation, 1)]
    for i, j in zip([nn.BatchNorm2d(output_activation), nn.ReLU(inplace=True)],[BN, activation]):
        if j==True:
            layer.append(i)
    return nn.Sequential(*layer)

In [4]:
def conv5x5_bn_relu(inp_activation, output_activation, BN = True, activation = True):
    "con 5x5 + Batchnormalization + relu"
    layer = [nn.Conv2d(inp_activation, output_activation, 5, padding = 2)]
    for i, j in zip([nn.BatchNorm2d(output_activation), nn.ReLU(inplace=True)], [BN, activation]):
        if j==True:
            layer.append(i)
    return nn.Sequential(*layer)

In [5]:
class bottleneck_block(nn.Module):
    
    def __init__(self, input_activation, list_filter=[256, 64, 256], down = None):
        super().__init__()
        self.conv1x1_1 = conv1x1_bn_relu(input_activation, list_filter[0])
        self.conv3x3 = conv3x3_bn_relu(list_filter[0], list_filter[1])
        self.conv1x1_2 = conv1x1_bn_relu(list_filter[1], list_filter[2])
        self.down = down
        if down!=None:
            self.contract_conv = down
    def forward(self , inp):
        x = inp
        c = self.conv1x1_1(inp)
        c = self.conv3x3(c)
        result = self.conv1x1_2(c)
        if self.down!=None:
            x = self.contract_conv(inp)
        
        out = result + x
        
        return out
            


In [7]:
class conv_block_bottle(nn.Module):
    
    def __init__(self, inp_activation, output):
        super().__init__()
        self.input = inp_activation
        self.output = output
        
        self.bottle1 = bottleneck_block(inp_activation, [inp_activation]*3)
            
        self.bottle2 = bottleneck_block(inp_activation, [inp_activation]*2 + [output], down  = nn.Conv2d(inp_activation
                                                                                                       ,output, 1))

    def forward(self , inp):
        c = self.bottle1(inp)
        c = self.bottle2(c)
        
        return c

In [2]:
m=Unet(2)